In [ ]:
from google.colab import drive
drive.mount('/content/drive/')



KeyboardInterrupt: 

In [ ]:

import glob
folder = '/content/drive/MyDrive/reddit/reddit clean'
folder_loc = glob.glob('{}/*.csv'.format(folder))
# first /content/drive/MyDrive/reddit/reddit clean/clean_reddit_comments_2019_12.csv
# last: /content/drive/MyDrive/reddit/reddit clean/clean_reddit_comments_2021_12.csv

['/content/drive/MyDrive/reddit/reddit clean/clean_reddit_comments_2019_12.csv',
 '/content/drive/MyDrive/reddit/reddit clean/clean_reddit_comments_2020_01.csv',
 '/content/drive/MyDrive/reddit/reddit clean/clean_reddit_comments_2020_02.csv',
 '/content/drive/MyDrive/reddit/reddit clean/clean_reddit_comments_2020_03.csv',
 '/content/drive/MyDrive/reddit/reddit clean/clean_reddit_comments_2020_04.csv',
 '/content/drive/MyDrive/reddit/reddit clean/clean_reddit_comments_2020_05.csv',
 '/content/drive/MyDrive/reddit/reddit clean/clean_reddit_comments_2020_06.csv',
 '/content/drive/MyDrive/reddit/reddit clean/clean_reddit_comments_2020_07.csv',
 '/content/drive/MyDrive/reddit/reddit clean/clean_reddit_comments_2020_08.csv',
 '/content/drive/MyDrive/reddit/reddit clean/clean_reddit_comments_2020_09.csv',
 '/content/drive/MyDrive/reddit/reddit clean/clean_reddit_comments_2020_10.csv',
 '/content/drive/MyDrive/reddit/reddit clean/clean_reddit_comments_2020_11.csv',
 '/content/drive/MyDrive/red

In [ ]:


import sklearn
import pandas as pd
import numpy as np
#from deep_translator import GoogleTranslator
import xgboost as xgb
import matplotlib.pyplot as plt
import re
#import text2vec

# Create an empty DataFrame with the specified columns

columns = ['body', 'id','subreddit', 'body_NO_PUNC', 'fentanyl',
'count_fentanyl', 'heroin', 'count_heroin', 'oxy', 'count_oxy', 'MDMA',
'count_MDMA', 'INHALANTS', 'count_INHALANTS', 'GHB', 'count_GHB',
'COCAINE', 'count_COCAINE', 'marijuana', 'count_marijuana',
'total_flags', 'match', 'total_text_no_punc', 'total_text',
'concat', 'total_no_drug', 'year']


balanced_sample = pd.DataFrame(columns=columns)
no_included_samples = pd.DataFrame(columns=columns)



def balanced_sample_maker(csv, balanced, left_overs, un_balancer):
    file_path_submission = csv
    columns_to_read = ['body', 'id','subreddit', 'body_NO_PUNC', 'fentanyl',
       'count_fentanyl', 'heroin', 'count_heroin', 'oxy', 'count_oxy', 'MDMA',
       'count_MDMA', 'INHALANTS', 'count_INHALANTS', 'GHB', 'count_GHB',
       'COCAINE', 'count_COCAINE', 'marijuana', 'count_marijuana',
       'total_flags', 'match']

    submissions = pd.read_csv(file_path_submission, usecols=columns_to_read)
    submissions['year'] = file_path_submission[-11:-7]

    #submissions['country'] = submissions['subreddit'].map(dictionary)
    #submissions = submissions[submissions['no_match'] == 0]
    #submissions['match'] = np.where(submissions['no_match'] == 0, 1,0)

    #submissions['match'] = submissions['no_match']
    submissions[submissions['body'] != '[removed]']

    submissions['body'] = submissions['body'].fillna(' ')

    submissions['body_NO_PUNC'] = submissions['body_NO_PUNC'].fillna(' ')

    #submissions change this to be rename
    submissions = submissions.rename(columns={"body_NO_PUNC": "total_text_no_punc", "body": "total_text"})
    #print(submissions.columns)

    submissions = submissions[~submissions['total_text'].str.contains('\[removed\]', case=False)]
    submissions = submissions[~submissions['total_text'].str.contains('\[deleted\]', case=False)]
    removed_pattern = re.compile(r'\[r(e|m)(o|0)v(e|a)ed\]', flags=re.IGNORECASE)
# Filter out rows containing '[removed]' variations in the 'total_text' column
    submissions = submissions[~submissions['total_text'].str.extract(removed_pattern).notna().any(axis=1)]

    lst = submissions.columns.tolist()

    # Splitting column names by commas and flattening the resulting list
    split_columns = [item.split(',') for item in lst]
    flattened_list = [item for sublist in split_columns for item in sublist]

    filtered_list = [item for item in flattened_list[flattened_list.index('total_text_no_punc') + 1:]]
    filtered_list = [item for item in filtered_list if "count_" not in item]
    filtered_list = [item for item in filtered_list[:filtered_list.index('total_flags'):]]

    submissions['concat'] = submissions[filtered_list].values.tolist()

    submissions['concat'] = submissions['concat'].apply(lambda x: [item for item in x if pd.notna(item)])

    submissions['concat'] = submissions['concat'].apply(lambda x: [item.strip() for sublist in x for item in sublist.split(',')] if x else [])

    def remove_occurrences(row):
        for item in row['concat']:
            # Create a regular expression pattern for whole-word match with case-insensitive flag
            pattern = r'\b{}\b'.format(re.escape(item))
            # Use re.sub to replace occurrences based on the pattern
            row['total_text_no_punc'] = re.sub(pattern, '', row['total_text_no_punc'], flags=re.IGNORECASE)
        return row['total_text_no_punc']

    # Apply the function to each row
    submissions['total_no_drug'] = submissions.apply(remove_occurrences, axis=1)

    def compbl(s):
        return ' '.join(s.split())

    submissions['total_no_drug'] = submissions['total_no_drug'].apply(compbl)


    drug_summary = submissions.columns.tolist()
    count_columns = [col for col in drug_summary if col.startswith('count_')]
    #submissions = submissions[submissions['total_text'].str.contains(re.escape('[removed]'))]
    n = int(round(submissions['match'].sum() * un_balancer))
    sample_no_drug = submissions[submissions['match'] == 0].sample(n)
    not_selected_samples = submissions[~submissions.index.isin(sample_no_drug.index)]
    #keeping not included samples
    sample_drug = submissions[submissions['match'] == 1]
    balanced = pd.concat([sample_drug,sample_no_drug], ignore_index = True)

    print("total drug related content {}, total length of balanced dataset {}".format(submissions['match'].sum(), len(balanced)))
    return balanced, not_selected_samples





for x in folder_loc:
    bal, ns = balanced_sample_maker(x,
                          balanced_sample,
                         no_included_samples,
                         1.3)
    no_included_samples = pd.concat([no_included_samples,ns], ignore_index = True)
    balanced_sample = pd.concat([balanced_sample, bal], ignore_index = True)

no_included_samples.to_csv('/content/drive/MyDrive/reddit/balanced_comment/no_include_sample_comm1.csv')


balanced_sample.to_csv('/content/drive/MyDrive/reddit/balanced_comment/balance_sample_comm1.csv')



total drug related content 9, total length of balanced dataset 21


<ipython-input-3-7723aba51b82>:33: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  submissions = pd.read_csv(file_path_submission, usecols=columns_to_read)


total drug related content 10768, total length of balanced dataset 24766


<ipython-input-3-7723aba51b82>:33: DtypeWarning: Columns (10,12,14,16,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  submissions = pd.read_csv(file_path_submission, usecols=columns_to_read)


total drug related content 10168, total length of balanced dataset 23386


...

total drug related content 20500, total length of balanced dataset 47150


<ipython-input-3-7723aba51b82>:33: DtypeWarning: Columns (10,12,14,16,18,20,22) have mixed types. Specify dtype option on import or set low_memory=False.
  submissions = pd.read_csv(file_path_submission, usecols=columns_to_read)


total drug related content 20559, total length of balanced dataset 47286


<ipython-input-3-7723aba51b82>:33: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  submissions = pd.read_csv(file_path_submission, usecols=columns_to_read)


total drug related content 19475, total length of balanced dataset 44793


NameError: name 'no_included_samples' is not defined

# New Section